In [17]:
import sys
import os

# Add the root directory to the Python environment
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [20]:
import requests
import json
import pandas as pd
from src import api
import random


In [19]:
def read_config(file_path="config.json"):
    with open(file_path, "r") as file:
        return json.load(file)

In [ ]:


for version in ["v2", "v3", "v4"]:
    config = read_config('/workspaces/uniswap-crawler/config_new.json')
    SUBGRAPH_URL = config['services']['graph_endpoints'][version]
    if version == "v2":
        VOLUMES = """
        dailyVolumeETH
        dailyVolumeUSD
        dailyVolumeUntracked
        """
    else:
        VOLUMES = """
        volumeETH
        volumeUSD
        volumeUSDUntracked
        """
    all_data = []
    skip = 0
    PAGE_SIZE = 1000

    DATE_GTE = 1704067200 # 2024-01-01
    DATE_LTE = 1751327999 # 2025-06-30

    while True:
        query = f"""
        {{
          uniswapDayDatas(
            where: {{ date_gte: {DATE_GTE}, date_lte: {DATE_LTE} }}
            first: {PAGE_SIZE}
            skip: {skip}
            orderBy: date
            orderDirection: asc
          ) {{
            id
            date
            {VOLUMES}
            txCount
          }}
        }}
        """

        result = api.the_graph_query_wrapper_by_endpoint(
            query,
            random.choice(config['services']['the_graph_tokens']),
            SUBGRAPH_URL
        )
        print(result)
        page_data = result["data"]["uniswapDayDatas"]

        print(f"Fetched {len(page_data)} items (skip={skip})")

        if not page_data:
            break

        all_data.extend(page_data)
        # Convert the list of dicts to a DataFrame and save as CSV
        df = pd.DataFrame(all_data)
        df.to_csv(f"uniswap_day_data_{version}.csv", index=False)
        print(f"Saved results to uniswap_day_data_{version}.csv")
        if len(page_data) < PAGE_SIZE:
            break
        skip += PAGE_SIZE


{'data': {'uniswapDayDatas': [{'dailyVolumeETH': '40940.16863184833033955012169469727', 'dailyVolumeUSD': '94546135.52243273422950145004840572', 'dailyVolumeUntracked': '124358449.2928197653255110022128785', 'date': 1704067200, 'id': '19723', 'txCount': '156380468'}, {'dailyVolumeETH': '60234.91880232577317219019052731976', 'dailyVolumeUSD': '143318311.6823425895887315863882347', 'dailyVolumeUntracked': '178980192.1692459872901058502631201', 'date': 1704153600, 'id': '19724', 'txCount': '156507181'}, {'dailyVolumeETH': '79179.41791593395160858902905774975', 'dailyVolumeUSD': '179383971.5461465463585367960820657', 'dailyVolumeUntracked': '224774465.603392388244979226494391', 'date': 1704240000, 'id': '19725', 'txCount': '156640907'}, {'dailyVolumeETH': '66546.52763779688605730318976536022', 'dailyVolumeUSD': '149653584.3600848949574455923224889', 'dailyVolumeUntracked': '185860430.9668491620983426560749889', 'date': 1704326400, 'id': '19726', 'txCount': '156780009'}, {'dailyVolumeETH': 

In [28]:
df.head()

,date,id,txCount,volumeETH,volumeUSD,volumeUSDUntracked
0,2025-01-24,20112,14,0.000670,2.210004,0
1,2025-01-26,20114,16,0.002346,7.740621,0
2,2025-01-27,20115,30,0.001209,3.988748,0
3,2025-01-28,20116,36,0.059730,197.112665,0
4,2025-01-29,20117,102,0.474931,1567.302691,0


In [ ]:
import glob

# Find all files matching the pattern
csv_files = glob.glob("uniswap_day_data_*.csv")
sum_avg = 0
for file in csv_files:
    print(f"Processing {file}...")
    df = pd.read_csv(file)

    # Try to find the volume column (could be volumeUSD, volumeETH, etc.)
    # We'll prefer volumeUSD if available
    volume_col = None
    for col in [ "dailyVolumeUSD", "volumeUSD"]:
        if col in df.columns:
            volume_col = col
            break

    if not volume_col:
        print(f"Could not find a volume column in {file}. Skipping.")
        continue

    # Convert date column to datetime if not already
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], unit="s")

    # Filter for 2024-01-01 to now
    start_date = pd.Timestamp("2025-06-01")
    end_date = pd.Timestamp.now()
    df_2024 = df[(df["date"] >= start_date) & (df["date"] <= end_date)]

    # Calculate total and average daily swap volume
    total_volume = df_2024[volume_col].astype(float).sum()
    num_days = (df_2024["date"].dt.date.nunique())
    avg_daily_volume = total_volume / num_days if num_days > 0 else 0

    print(f"File: {file}")
    print(f"  Total swap volume (2024 to June 30): {total_volume:,.2f}")
    print(f"  Average daily swap volume (2024 to June 30): {avg_daily_volume:,.2f}")
    print(f"  Number of days: {num_days}")
    sum_avg += avg_daily_volume
print(f"  sum: {sum_avg}")


Processing uniswap_day_data_v2.csv...
File: uniswap_day_data_v2.csv
  Total swap volume (2024 to June 30): 11,197,098,623.54
  Average daily swap volume (2024 to June 30): 373,236,620.78
  Number of days: 30
Processing uniswap_day_data_v3.csv...
File: uniswap_day_data_v3.csv
  Total swap volume (2024 to June 30): 18,534,118,112.90
  Average daily swap volume (2024 to June 30): 617,803,937.10
  Number of days: 30
Processing uniswap_day_data_v4.csv...
File: uniswap_day_data_v4.csv
  Total swap volume (2024 to June 30): 9,426,189,515.05
  Average daily swap volume (2024 to June 30): 314,206,317.17
  Number of days: 30
  sum: 1305246875.0496917


In [ ]:
import json

# Read and parse the defilama.json file
defilama_json_path = "/workspacess/uniswap-crawler/data/DefiLama/defilama.json"
with open(defilama_json_path, "r") as f:
    defilama_data = json.load(f)

# Optionally, display the parsed data (for inspection)
print(defilama_data)


{'id': 'parent#uniswap', 'name': 'Uniswap', 'url': 'https://uniswap.org/', 'description': 'Swap, earn, and build on the leading decentralized crypto trading protocol.', 'logo': 'https://icons.llama.fi/uniswap.png', 'gecko_id': 'uniswap', 'cmcId': '7083', 'chains': ['Ethereum', 'Arbitrum', 'Polygon', 'Base', 'BSC', 'Unichain', 'Avalanche', 'World Chain', 'OP Mainnet', 'ZKsync Era', 'Sei', 'Taiko', 'Scroll', 'Rootstock', 'Filecoin', 'Boba', 'Moonbeam', 'Manta', 'Mantle', 'Linea', 'Polygon zkEVM', 'Blast', 'Gnosis', 'BOB', 'Lisk', 'Corn', 'Goat', 'Hemi', 'Saga', 'XDC', 'LightLink', 'Lens', 'Telos', 'Celo', 'Zora', 'Ink', 'Soneium'], 'twitter': 'Uniswap', 'treasury': 'uniswap.js', 'governanceID': ['snapshot:uniswap', 'eip155:1:0x408ED6354d4973f66138C91495F2f2FCbd8724C3', 'compound:ethereum:0x5e4be8bc9637f0eaa1a755019e06a68ce081d58f'], 'github': ['Uniswap'], 'symbol': 'UNI', 'address': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984', 'childProtocols': [{}, {}, {}, {}], 'linkedProtocols': ['Uni

In [36]:
import pandas as pd

# Extract the relevant data from totalDataChartBreakdown
rows = []
for entry in defilama_data.get("totalDataChartBreakdown", []):
    timestamp = entry[0]
    breakdown = entry[1]
    eth_data = breakdown.get("Ethereum")
    if eth_data:
        row = {"timestamp": timestamp}
        # Add each Uniswap version's value if present
        for version in ["Uniswap V1", "Uniswap V2", "Uniswap V3"]:
            row[version] = eth_data.get(version, 0)
        rows.append(row)

# Create DataFrame
df_eth = pd.DataFrame(rows)

# Optionally, convert timestamp to datetime
df_eth["date"] = pd.to_datetime(df_eth["timestamp"], unit="s")

# Display the DataFrame
print(df_eth)


       timestamp  Uniswap V1  Uniswap V2  Uniswap V3       date
0     1541116800        1688           0           0 2018-11-02
1     1541203200        2537           0           0 2018-11-03
2     1541289600        2537           0           0 2018-11-04
3     1541376000         204           0           0 2018-11-05
4     1541462400         793           0           0 2018-11-06
...          ...         ...         ...         ...        ...
2437  1751673600       42609   324074177   218881183 2025-07-05
2438  1751760000       39162   166283589   315079403 2025-07-06
2439  1751846400       45741   128137724   472504076 2025-07-07
2440  1751932800       62488    46714461   448076509 2025-07-08
2441  1752019200       53134           0           0 2025-07-09

[2442 rows x 5 columns]


In [37]:
# Add a new column that is the sum of all Uniswap versions
df_eth["Uniswap_Total"] = df_eth[["Uniswap V1", "Uniswap V2", "Uniswap V3"]].sum(axis=1)

# Optionally, display the updated DataFrame
print(df_eth)


       timestamp  Uniswap V1  Uniswap V2  Uniswap V3       date  Uniswap_Total
0     1541116800        1688           0           0 2018-11-02           1688
1     1541203200        2537           0           0 2018-11-03           2537
2     1541289600        2537           0           0 2018-11-04           2537
3     1541376000         204           0           0 2018-11-05            204
4     1541462400         793           0           0 2018-11-06            793
...          ...         ...         ...         ...        ...            ...
2437  1751673600       42609   324074177   218881183 2025-07-05      542997969
2438  1751760000       39162   166283589   315079403 2025-07-06      481402154
2439  1751846400       45741   128137724   472504076 2025-07-07      600687541
2440  1751932800       62488    46714461   448076509 2025-07-08      494853458
2441  1752019200       53134           0           0 2025-07-09          53134

[2442 rows x 6 columns]


In [49]:
# Filter the DataFrame for dates between January 2024 and June 2025
start_date = pd.to_datetime("2025-06-08")
end_date = pd.to_datetime("2025-07-08")
mask = (df_eth["date"] >= start_date) & (df_eth["date"] <= end_date)
df_filtered = df_eth.loc[mask]

# Calculate the average daily Uniswap_Total
average_daily_uniswap_total = df_filtered["Uniswap_Total"].mean()

# Calculate the total sum of Uniswap_Total in the period
total_uniswap_sum = df_filtered["Uniswap_Total"].sum()

print(f"Average daily Uniswap_Total (Jan 2024 - Jun 2025): {average_daily_uniswap_total}")
print(f"Total Uniswap_Total sum (Jan 2024 - Jun 2025): {total_uniswap_sum}")


Average daily Uniswap_Total (Jan 2024 - Jun 2025): 833845587.1612903
Total Uniswap_Total sum (Jan 2024 - Jun 2025): 25849213202
